### Use AzureOpenAI

In [1]:
from azureopenai import get_azureopenai_llm, get_azureopenai_embeddings

llm = get_azureopenai_llm()

In [30]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith, as a language model, can help with testing in various ways:\n\n1. Test case generation: Langsmith can assist in generating test cases based on user-defined criteria or specifications. By providing input conditions, constraints, and expected output, Langsmith can generate a wide range of test cases to cover different scenarios.\n\n2. Test data generation: Langsmith can create realistic and diverse test data to simulate various scenarios. This can be helpful in testing the robustness and scalability of software systems.\n\n3. Test automation: Langsmith can assist in automating the testing process by generating code snippets or test scripts. This can help in automating repetitive testing tasks, reducing manual effort, and improving overall testing efficiency.\n\n4. Test documentation: Langsmith can help in creating test documentation, such as test plans, test cases, and test reports. By providing clear and concise descriptions, Langsmith can assist in documen

### Run this to use phi3 instead

In [24]:
from ollama import get_ollama_llm
llm = get_ollama_llm()

In [25]:
llm.invoke("Tell me a joke")

'Why don\'t scientists trust atoms? Because they make up everything!\n\n(Note: Humor can vary widely, and what is considered funny to one person might not be to another. This joke plays on the double meaning of "make up" - both literally in chemistry and figuratively.)'

### Integrating System Prompts

In [2]:
# Prompt templates allow passing in a list of messages to be used as a template for the prompt.
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

In [3]:
from langchain_core.output_parsers import StrOutputParser

# The output parser is used to extract the output from the response.
output_parser = StrOutputParser()

# The chain is the sequence of components that will be used to generate the response.
chain = prompt | llm | output_parser

In [4]:
# The result is not precise because the model is not trained on this topic too well??
# AzureOpenAI answers something vague, phi3 allucinates.
result = chain.invoke({"input": "how can langsmith help with testing?"})
print(result)

Langsmith can greatly assist with testing by providing a powerful and flexible platform for automating and executing test cases. Here are some ways in which Langsmith can help with testing:

1. Test Case Creation: Langsmith allows testers to easily create test cases using its intuitive and user-friendly interface. Test cases can be created using a simple scripting language or by recording user interactions with the system under test.

2. Test Execution: Langsmith provides a robust and scalable execution environment for running test cases. It supports parallel execution across multiple machines, allowing for faster and more efficient testing. Test results are automatically collected and can be easily analyzed.

3. Test Data Generation: Langsmith includes built-in capabilities for generating test data, making it easier to create comprehensive and realistic test scenarios. This helps ensure that the system under test is thoroughly tested in various scenarios.

4. Test Management: Langsmit

### Using RAG

In [16]:
# Retrival is useful when you have too much data to pass to the prompt directly.
# You just want to pre-select the relevant data and pass it to the model.
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/old/user_guide")

docs = loader.load()
docs

[Document(page_content="\n\n\n\n\nLangSmith User Guide | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly un

In [17]:
embeddings = get_azureopenai_embeddings()

In [20]:
# FAISS is a vector store that allows you to store and retrieve vectors.
# You can use write_index to store the db to disk and read_index to retrieve it.
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
documents

[Document(page_content='LangSmith User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/old/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.', 'language': 'en'}),
 Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a plat

In [8]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

# Create a chain for passing a list of Documents to the model.
# The prompt must have a {context} placeholder where the context will be inserted.
document_chain = create_stuff_documents_chain(llm, prompt)

In [22]:
from langchain_core.documents import Document

# Here we pass the context as a list of Documents.
document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing users to visualize test results.'

In [23]:
from langchain.chains import create_retrieval_chain

# Retriever is used to retrieve the relevant documents from the vector store.
# The retrieval chain is used to pass the retrieved documents to the model.
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [24]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith can help with testing by providing several features and workflows that facilitate the development and evaluation of LLM (Language Model) applications. These include:

1. Prototyping: LangSmith allows for quick experimentation between prompts, model types, retrieval strategies, and other parameters, making it easier to prototype LLM applications.

2. Debugging: LangSmith provides clear visibility and debugging information at each step of an LLM sequence, making it easier to identify and resolve issues. It allows developers to enable tracing, which helps in debugging when unexpected results or errors occur.

3. Initial Test Set: Developers can create datasets consisting of inputs and reference outputs to run tests on their LLM applications. These test cases can be uploaded in bulk, created on the fly, or exported from application traces. LangSmith also supports custom evaluations to score test results.

4. Comparison View: LangSmith offers a user-friendly comparison view for te

In [25]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query
# The prompt must have a {chat_history} placeholder where the chat history will be inserted.

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
# Remember the retriever is the chain to retrieve the relevant documents from the vector store.
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [26]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is fa

In [17]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

# The retrieval chain is used to pass the retrieved documents to the model with history.
# The document chain is used to pass the context.
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [27]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='meaning that they involve a series of interactions between the user and the application. LangSmith provides a threads view that groups traces from a single conversation together, making it easier to track the performance of and annotate your application across multiple turns.Was this page helpful?You can leave detailed feedback on GitHub.PreviousQuick StartNextOverviewPrototypingBeta TestingProductionCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2024 LangChain, Inc.', metadata={'source': 'https://docs.smith.langchain.com/old/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth o

In [28]:
from langchain.tools.retriever import create_retriever_tool

# Make the previous retriever chain a tool so that it can be used in the chat.
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [29]:
# https://app.tavily.com/home
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

tools = [retriever_tool, search]

In [30]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

prompt = hub.pull("hwchase17/openai-functions-agent")

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [31]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...
LangSmith can help with testing by providing various services and tools. Here are a few ways LangSmith can assist with testing:

1. Test Data Generation: LangSmith can generate realistic and diverse test data to ensure comprehensive testing of software applications. It can create synthetic data sets based on specific requirements, allowing testers to cover different test scenarios.

2. Test Case Generation: LangSmith can automatically generate test cases based on the specifications and requirements of the software being tested. This helps testers ensure that all possible scenarios and edge cases are covered during testing.

3. Test Automation: LangSmith can automate the execution of test cases, reducing the manual effort required for testing. It can integrate with testing frameworks and tools to execute tests automatically and provide detailed test reports.

4. Test Environment Provisioning: LangSmith can help provision test environments quickly 

{'input': 'how can langsmith help with testing?',
 'output': "LangSmith can help with testing by providing various services and tools. Here are a few ways LangSmith can assist with testing:\n\n1. Test Data Generation: LangSmith can generate realistic and diverse test data to ensure comprehensive testing of software applications. It can create synthetic data sets based on specific requirements, allowing testers to cover different test scenarios.\n\n2. Test Case Generation: LangSmith can automatically generate test cases based on the specifications and requirements of the software being tested. This helps testers ensure that all possible scenarios and edge cases are covered during testing.\n\n3. Test Automation: LangSmith can automate the execution of test cases, reducing the manual effort required for testing. It can integrate with testing frameworks and tools to execute tests automatically and provide detailed test reports.\n\n4. Test Environment Provisioning: LangSmith can help provis

In [32]:
agent_executor.invoke({"input": "what is the weather today in Javea, Spain?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather today in Javea, Spain'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Javea', 'region': 'Castilla-La Mancha', 'country': 'Spain', 'lat': 38.78, 'lon': 0.17, 'tz_id': 'Europe/Madrid', 'localtime_epoch': 1716360502, 'localtime': '2024-05-22 8:48'}, 'current': {'last_updated_epoch': 1716360300, 'last_updated': '2024-05-22 08:45', 'temp_c': 20.3, 'temp_f': 68.5, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 2.5, 'wind_kph': 4.0, 'wind_degree': 348, 'wind_dir': 'NNW', 'pressure_mb': 1017.0, 'pressure_in': 30.02, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 56, 'cloud': 5, 'feelslike_c': 20.3, 'feelslike_f': 68.5, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 6.0, 'gust_mph': 3.4, 'gust_kph': 5.4}}"}, {'url': 'https://www.aemet.es/en/eltiempo/prediccion/municipios/javea-xabia-id

{'input': 'what is the weather today in Javea, Spain?',
 'output': 'The current weather in Javea, Spain is sunny with a temperature of 20.3°C (68.5°F). The wind is blowing from the NNW direction at a speed of 4.0 kph (2.5 mph). The humidity is 56% and there are 5% clouds in the sky. The visibility is 10.0 km (6.0 miles) and the UV index is 6.0.'}

In [33]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...
LangSmith can help test your LLM applications in the following ways:

1. Personal Statement Review: LangSmith can review your personal statement and provide feedback on its clarity, organization, and content. They can also suggest improvements to make your personal statement more compelling and impactful.

2. Resume/CV Review: LangSmith can review your resume or CV and provide feedback on its format, structure, and content. They can offer suggestions to highlight your relevant experiences and skills effectively.

3. Letter of Recommendation Guidance: LangSmith can provide guidance on selecting the right recommenders for your LLM applications and offer advice on how to approach them for a strong letter of recommendation.

4. Application Strategy: LangSmith can help you develop a comprehensive strategy for your LLM applications, including selecting the right law schools, organizing your application materials, and prioritizing your application deadl

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': "LangSmith can help test your LLM applications in the following ways:\n\n1. Personal Statement Review: LangSmith can review your personal statement and provide feedback on its clarity, organization, and content. They can also suggest improvements to make your personal statement more compelling and impactful.\n\n2. Resume/CV Review: LangSmith can review your resume or CV and provide feedback on its format, structure, and content. They can offer suggestions to highlight your relevant experiences and skills effectively.\n\n3. Letter of Recommendation Guidance: LangSmith can provide guidance on selecting the right recommenders for your LLM applications and offer advice on how to approach them for a strong letter of recommendation.\n\n4. Application Strategy: LangSmith can help you develop a comprehensive strategy for your LLM applications